# Notebook to test the predictions

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import sys
from pathlib import Path

ML_DIR = Path().resolve().parents[0]
sys.path.append(str(ML_DIR))

# Import ML functions
from update_rolling import update_rolling_stats
from predict import predict_points

load_dotenv()
DATABASE_URL = os.getenv("ML_DATABASE_URL")
engine = create_engine(DATABASE_URL)


In [3]:
df_latest_rolling = update_rolling_stats(engine)

print("Latest rolling rows per player:")
display(df_latest_rolling.head())


No new games to update.
Latest rolling rows per player:


,player_id,game_id,game_date,matchup,team_abbreviation,minutes,points,assists,rebounds,steals,blocks,turnovers,avg_points_last5,avg_assists_last5,avg_rebounds_last5,avg_minutes_last5
0,2544,22500607,2026-01-18,LAL vs. TOR,LAL,32.0,24.0,7.0,4.0,0.0,2.0,3.0,25.2,6.8,7.0,32.6
1,101108,22500313,2025-12-01,LAC @ MIA,NaN,15.0,8.0,3.0,0.0,0.0,0.0,1.0,3.8,2.4,1.2,13.4
2,200768,22500560,2026-01-12,PHI @ TOR,PHI,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,1.2,0.6,8.2
3,201142,22500591,2026-01-16,HOU vs. MIN,HOU,40.0,39.0,7.0,4.0,2.0,1.0,5.0,27.8,4.8,7.4,38.8
4,201143,22500584,2026-01-15,GSW vs. NYK,GSW,21.0,5.0,5.0,5.0,1.0,2.0,1.0,6.6,3.8,6.0,18.4


## Run predictions

In [10]:
df_preds = predict_points(engine)

print("Predictions:")
df_preds = df_preds.sort_values("game_date", ascending=False)
display(df_preds.head(20))


Predictions:


,player_id,team_abbreviation,matchup,game_date,pred_points
0,2544,LAL,LAL vs. TOR,2026-01-18,19.406025
190,1630173,SAC,SAC vs. WAS,2026-01-16,11.753265
193,1630178,PHI,PHI vs. CLE,2026-01-16,28.248419
196,1630183,MIN,MIN @ HOU,2026-01-16,15.593656
197,1630188,None,CHI @ BKN,2026-01-16,11.200791
203,1630200,CHI,CHI @ BKN,2026-01-16,15.476953
212,1630245,CHI,CHI @ BKN,2026-01-16,9.653853
214,1630256,None,HOU vs. MIN,2026-01-16,3.007443
215,1630264,WAS,WAS @ SAC,2026-01-16,1.941283
221,1630530,None,NOP @ IND,2026-01-16,28.049080
